In [1]:
# Installing Huggingface transformers (and a few other packages that might be needed
# for small and quantized models)
!pip install -q transformers accelerate bitsandbytes optimum auto-gptq tiktoken blobfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.

In [3]:
from google.colab import drive
import pandas as pd

# Mounting file-system from drive
drive.mount('/content/drive')

# Function to load files
def load_file(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()
  return content

# Logining in and authenticating hugging face account
!pip install -q huggingface_hub
from huggingface_hub import login
login(token=load_file('/content/drive/My Drive/work/cimatec/enade_to_edag/data/keys/hf').strip())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Loading LLM (we picked a small model that fits T4 for google colab)
# model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# model_name = "TucanoBR/Tucano-2b4-instruct"
model_name = "meta-llama/Llama-3.2-3B-Instruct"
# model_name = "meta-llama/Llama-4-Scout-17B-16E-Instruct"


# Getting device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", \
                                             trust_remote_code=True, torch_dtype=torch.float16)

# Creating pipeline for text generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto", \
                     return_full_text=False, pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Function to ask model to clean the question
def question_cleaning(generator, original_question, temperature=0.3, max_new_tokens=512):
  prompt = f"""[INSTRUÇÕES GERAIS]
- Gera uma versão limpa da questão abaixo.
- Remova ruídos como textos inválidos, palavras malformadas e repetições desnecessárias, sem alterar o conteúdo, estrutura ou formato da questão.
- Não inclua nenhum raciocínio ou metadados na saída.

[QUESTÃO ORIGINAL]
{original_question}
"""

  response = generator(prompt, max_new_tokens=max_new_tokens, temperature=temperature, do_sample=True)[0]['generated_text']
  return response.strip()

# Function to create questions based on a command given the original question and
# a desired format
def question_generation(generator, clean_question, target_format, \
                 temperature=0.3, max_new_tokens=512):
  prompt = f"""[INSTRUÇÕES GERAIS]
- Faça uma nova versão dessa questão com o formato especificado.
- Use apenas o texto abaixo como guia.
- Não inclua nenhum raciocínio, metadados ou repetição da questão original.
- A saída deve obedecer estritamente ao FORMATO DE SAÍDA.

[QUESTÃO ORIGINAL]
{clean_question}

[FORMATO DE SAÍDA]
{target_format}
"""

  response = generator(prompt, max_new_tokens=max_new_tokens, temperature=temperature, do_sample=True)[0]['generated_text']
  return response.strip()

In [6]:
# Instantiating to test pipeline
question_year = 2023
question_number = 1
question_type = "closed"
question_path = f"/content/drive/My Drive/work/cimatec/enade_to_edag/data/prova_{question_year}/{question_type}_question_{question_number:02d}.txt"
original_question = load_file(question_path)

target_format_type = "resposta_unica"
target_format_path = f"/content/drive/My Drive/work/cimatec/enade_to_edag/data/edag_question_formats/{target_format_type}.txt"
target_format = load_file(target_format_path)

clean_question = question_cleaning(generator, original_question)
new_question = question_generation(generator, clean_question, target_format)

print(f"\n\n********** Original Question **********\n")
print(original_question)

print(f"\n\n********** Clean Question **********\n")
print(clean_question)

print("\n\n********** New Question **********\n")
print(new_question)



********** Original Question **********

questão 01
A fome e a insegurança alimentar, anti gos problemas da sociedade, são agravados em regiões com 
elevados índices de desigualdade social. Propor soluções para esse quadro requer uma abordagem 
multi dimensional, que possibilite a interação entre as dimensões sociais, culturais, políti cas, econômicas 
e ambientais envolvidas na produção e na distribuição de alimentos.
Foto: Unicef/Sayed Bidel. Impacto dos confl itos sobre 
pessoas mais vulneráveis.
Foto: FAO/Anatolii Stepanov. Colheita de trigo perto 
da vila de Krasne, na Ucrânia.
       
 Foto: Unicef/Safi dy Andriananten. Secas em Madagascar colocam o país africano entre aqueles onde há mais fome.
 Disponível em: htt ps://news.un.org/pt/story/2022/05/1788102. Acesso em: 10 jun. 2023.
 Considerando o texto e as imagens apresentados, avalie as asserções a seguir e a relação proposta entre elas.
I.  A fome no mundo é um fenômeno biológico e sociológico inevitável.
 PORQUE
II.  A dis